In [70]:
# 优化参数
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
import random



"""加载数据集"""

train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

"""检测设备"""
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"


"""组建神经网络"""
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            # nn.Linear(128, 64),
            # nn.ReLU(),
            # nn.Linear(64, 32),
            # nn.ReLU(),
            # nn.Linear(32, 16),
            # nn.ReLU(),
            # nn.Linear(16, 10)
            nn.Linear(128, 10)
        )


    def forward(self, x):
        x = self.flatten(x)
        logits = self.stack(x)
        return logits

"""将模型转移到设备上"""
model = NeuralNetwork().to(device)

model.load_state_dict(torch.load('PyTorch_code_7_pth/model_1.pth', weights_only=True))

"""生成迭代器DataLoader"""
# 批次大小
batch_size = 60

train_DataLoader = DataLoader(dataset=train_data, batch_size=batch_size)
test_DataLoader = DataLoader(dataset=test_data, batch_size=batch_size)

"""定义损失函数和优化器"""
loss_def = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.0001)

# 训练轮数
epochs = 50

# 封装训练函数
def train(model, dataloader, loss_def, optim):
    data_size = len(dataloader.dataset)
    model.train()
    count = 0
    for Batch, (Images, Labels) in enumerate(dataloader):
        Images, Labels = Images.to(device), Labels.to(device)
        logits = model(Images)
        loss = loss_def(logits, Labels)
        loss.backward()
        count += 1
        if count % 4 == 0:
            optim.step()
            optim.zero_grad()
        if (Batch + 1) % 100 == 0:
            loss, current = loss.item(), (Batch + 1) * len(Images)
            print(f"Loss: {loss:>6f}   |   [current: {current:>5d}/{data_size}]")
    return


# 封装验证函数
def test(model, dataloader, loss_def):
    data_size = len(dataloader.dataset)
    batchs = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for batch, (Image, Label) in enumerate(dataloader):
            Image, Label = Image.to(device), Label.to(device)
            logits = model(Image)
            test_loss += loss_def(logits, Label)
            correct += (logits.argmax(1) == Label).type(torch.float).sum().item()
    result = correct / data_size
    batch_loss = test_loss / batchs
    print(f"Test Error:\ntest_correct: {result:>.6f}   |   [test_loss: {batch_loss:>.6f}]")
    return


# # 进行循环多轮训练
# for index in range(epochs):
#     print(f"Epochs {index + 1}" + "\n+" + "-" * 80)
#     train(model, train_DataLoader, loss_def, optim)
#     test(model, test_DataLoader, loss_def)


# 保存模型训练参数
# torch.save(model.state_dict(), "PyTorch_code_7_pth/model_1.pth")

# 加载模型参数
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("PyTorch_code_7_pth/model_1.pth", weights_only=True))


# 进行预测
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

rand = random.randint(0, 1000)

model.eval()
Image, Label = test_data[rand][0], test_data[rand][1]
with torch.no_grad():
    Image = Image.to(device)
    predict = model(Image)
    print(predict)
    print(predict[0].argmax(0))
    predicted, actual = classes[predict[0].argmax(0)], classes[Label]

print(f"Predicted: {predicted}   |   Actual: {actual}")




tensor([[  0.9997,  -0.1041,   7.4937,   2.7053,  10.7942,  -6.5967,   7.1819,
         -17.7987,   3.9923, -12.9041]])
tensor(4)
Predicted: Coat   |   Actual: Coat
